In [ ]:
from openff.toolkit import Topology, Molecule, ForceField
from openff.interchange import Interchange
from openff.utilities import get_data_file_path
import MDAnalysis as mda
u = mda.Universe("solvated_complex_mg.pdb")
sel_mg = u.select_atoms("(element Mg)")
#sel_solv = u.select_atoms("(element Mg Na Cl) or (resname HOH)")
sel_everything_else = u.select_atoms("not(element Mg)")
#sel_everything_else = u.select_atoms("not((element Mg Na Cl) or (resname HOH))")
#print(sel_mg, sel_everything_else)
sel_mg.write("mg.pdb")
sel_everything_else.write("everything_else.pdb")

In [ ]:
top = Topology.from_pdb("everything_else.pdb",
                        unique_molecules=[Molecule.from_smiles("CS(=O)(=O)c1ccc(c2c1[C@@H](C(C2)(F)F)O)Oc3cc(cc(c3)F)C#N")]
                        )
sage_ff14sb = ForceField("openff-2.2.1.offxml", "ff14sb_off_impropers_0.0.4.offxml")
interchange_everything_else = sage_ff14sb.create_interchange(top)

In [ ]:
# Parameterize the Mg using openmmforcefields and the amber tip3p_HFE_multivalent FF
from openmm import app
import openmm

pdb = app.PDBFile("mg.pdb")
# Load force field including multivalent ions
forcefield = app.ForceField("amber/tip3p_HFE_multivalent.xml")

# Create system
# We put some specific nonbonded settings here to ensure the resulting system is combineable 
# with the components we're parameterizing with Sage and our Amber FF14SB port
system = forcefield.createSystem(pdb.topology, 
                                 nonbondedMethod=app.PME,
                                 constraints=app.HBonds,
                                 nonbondedCutoff=0.9*openmm.unit.nanometer,
                                switchDistance=0.8*openmm.unit.nanometer)
# Currently, Interchange.from_openmm requires there to be _some_ force for these, even if they aren't used. 
system.addForce(openmm.HarmonicBondForce())
system.addForce(openmm.HarmonicAngleForce())
system.addForce(openmm.PeriodicTorsionForce())

In [ ]:
%env INTERCHANGE_EXPERIMENTAL=1
interchange_solv = Interchange.from_openmm(system, pdb.topology, positions=pdb.positions)

In [ ]:
interchange_all = interchange_everything_else.combine(interchange_solv)

In [ ]:
# Construct and configure a Langevin integrator at 300 K with an appropriate friction constant and time-step
integrator = openmm.LangevinIntegrator(
    300 * openmm.unit.kelvin,
    1 / openmm.unit.picosecond,
    0.002 * openmm.unit.picoseconds,
)

# Under the hood, this creates *OpenMM* `System` and `Topology` objects, then combines them together
simulation = interchange_all.to_openmm_simulation(integrator=integrator)

# Add a reporter to record the structure every 100 steps, 1000 if using a CUDA-enabled GPU
try:
    openmm.Platform.getPlatformByName("CUDA")
    stride = 1000
except openmm.OpenMMException:
    stride = 100

dcd_reporter = openmm.app.DCDReporter(file="trajectory.dcd", reportInterval=stride)
simulation.reporters.append(dcd_reporter)

In [ ]:
simulation.context.setVelocitiesToTemperature(300 * openmm.unit.kelvin)
simulation.runForClockTime(1.0 * openmm.unit.minute)

In [ ]:
import mdtraj
import nglview
trajectory: mdtraj.Trajectory = mdtraj.load(
    "trajectory.dcd", top=mdtraj.Topology.from_openmm(interchange_all.to_openmm_topology())
)

view = nglview.show_mdtraj(trajectory.image_molecules())
view.add_representation("line", selection="protein or water")
# Visualize the Mgs
view.add_representation("spacefill", selection="not protein and not water")

view